# Creación de BBDD

In [ ]:
import sqlite3

def crea_bbdd(nombre_tabla, columnas):
    mi_conexion = sqlite3.connect('Biblioteca.db')
    mi_cursor = mi_conexion.cursor()

    # Construir la parte de la consulta SQL para las columnas
    columnas_sql = ', '.join([f'{key} {value}' for key, value in columnas.items()])
    
    # Crear la tabla
    mi_cursor.execute(f'''
        CREATE TABLE {nombre_tabla} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            {columnas_sql}
        )
    ''')
    
    mi_conexion.commit()
    mi_conexion.close()



def crea_diccionario_columnas():
    diccionario_columnas = {}
    
    cantidad_elementos = int(input('Introduce la cantidad de elementos: '))
    
    for _ in range(cantidad_elementos):
        nombre_columna = input('Introduce el nombre de la columna: ')
        tipo_columna = input(f'Introduce el tipo de la columna {nombre_columna} (por ejemplo, VARCHAR, INTEGER): ')
        diccionario_columnas[nombre_columna] = tipo_columna
    
    return diccionario_columnas

def inserta_datos(nombre_tabla, columnas):
    mi_conexion = sqlite3.connect('Biblioteca.db')
    mi_cursor = mi_conexion.cursor()

    valores = tuple(input(f'Introduce {key}: ') for key in columnas.keys())
    mi_cursor.execute(f'INSERT INTO {nombre_tabla} VALUES (NULL, {", ".join(["?"] * len(columnas))})', valores)

    mi_conexion.commit()
    mi_conexion.close()

    
if __name__ == "__main__":

    nombre_tabla = input('Introduce nombre de la tabla: ')
    diccionario_columnas=crea_diccionario_columnas()
    crea_bbdd(nombre_tabla, diccionario_columnas)
    insert=input('¿Quieres introducir datos en la tabla creada? ')
    if insert.lower() == 'si':
        while True:
            inserta_datos(nombre_tabla, diccionario_columnas)
            seguir = input('Quieres introducir más datos (si/no): ')
            if seguir.lower() != 'si':
                break
    